# Open-weather meteo

On peut supposer que ce qui implique les retards sont :

precipitations
- `precipitations`
- `rain`
- `snow` 

conditions difficiles
- `wind_speed_10m`
- `wind_gusts_10m`

temperature
- `temprature_2m`

visibilité
- `cloud_cover`
- `weather_code`

ajoute variables "a priori inutiles" pour
- vérifier que le modèle surapprend pas
- features de contrôles
- mieux interpréter l'importance des variables
- détecter d'éventuelles corrélations inattendues

- rayonnement solaire `shortwave_radiation`
- point de rosée `dew_point_2m`
- direction du vent `wind_direction_10m`
- indice uv `uv_index`

intervalle de temps : 1h

In [1]:
%pip install openmeteo-requests
%pip install requests-cache retry-requests numpy pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
from datetime import datetime, timedelta
import json
# import os
# import boto3

In [47]:
def get_dates(mode="archive"):
    today = datetime.now()
    if mode == "archive":
        # Pour l'historique massif : du 1er jan 2023 à J-1
        start = "2023-01-01"
        end = (today - timedelta(days=1)).strftime('%Y-%m-%d')
    else:
        # Pour le focus "Aujourd'hui" : J-1 et Today
        start = (today).strftime('%Y-%m-%d')
        end = (today + timedelta(days=7)).strftime('%Y-%m-%d')
    return start, end

def fetch_weather_history(latitude, longitude, outfile, mode="archive"):
    
    # 1. Récupération des dates dynamiques
    start_d, end_d = get_dates(mode)
    
    # 2. Sélection de l'URL appropriée
    if mode == "archive":
        url = "https://archive-api.open-meteo.com/v1/archive"
    else:
        url = "https://api.open-meteo.com/v1/forecast"
    
    # 3. Configuration des paramètres avec les dates corrigées
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_d,
        "end_date": end_d,
        "hourly": [
            "temperature_2m", # Temperature à 2m du sol (en °C)
            "precipitation", # Cumul total d'eau pluie + neige fondue durant l'heure(en mm)
            "rain", # Quantité de pluie tombée durant l'heure (en mm)
            "wind_speed_10m", # Vitesse du vent à 10m du sol (en km/h)
            "snowfall", # Quantité de neige tombée durant l'heure (en cm)
            "wind_gusts_10m", # Rafales de vent à 10m du sol durant l'heure (en km/h)
            "weather_code", # code météo (voir tableau des codes météo plus bas)
            "cloud_cover", # Couverture nuageuse totale (en %) -- 0 = ciel dégagé, 100 = couvert
            "shortwave_radiation", # Rayonnement solaire à la surface durant l'heure (en W/m²)
            "dew_point_2m", # Point de rosée à 2m du sol (en °C)
            "wind_direction_10m" # Direction du vent à 10m du sol (en degrés)
        ],
        "daily": ["sunrise", "sunset"],
        "timezone": "Europe/Stockholm"
    }

    # 4. Requête et traitement
    r = requests.get(url, params=params)
    r.raise_for_status()

    # 5. Sauvegarde brute (raw)
    raw_data = r.json()

    # 6. Sauvegarde en JSON pour garder la structure exacte de l'API
    with open(outfile, 'w', encoding='utf-8') as f:
        json.dump(raw_data, f, ensure_ascii=False, indent=4)
        
    print(f" Données BRUTES ({mode}) sauvegardées dans : {outfile}")

'''def upload_to_s3(local_file: str, bucket: str, key: str):
    """Upload un fichier local sur S3"""
    s3 = boto3.client(
        "s3",
        aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
        region_name=os.getenv("AWS_REGION")
    )
    try:
        s3.upload_file(local_file, bucket, key)
        print(f"Upload réussi vers s3://{bucket}/{key}")
    except Exception as e:
        print("Erreur upload S3 :", e)'''

# --- EXÉCUTION ---
# Pour historique global (à mettre à jour tous les deux jours)
fetch_weather_history(
    latitude=59.3251172,
    longitude=18.0710935,
    mode="archive",
    outfile="weather_stockholm_archive.json"
)

# Pour ton appel "Temps réel / Forecast" (à lancer chaque jour)
fetch_weather_history(
     latitude=59.3251172, 
     longitude=18.0710935, 
     mode="forecast", 
     outfile="weather_stockholm_today+7j.json"
)

 Données BRUTES (archive) sauvegardées dans : weather_stockholm_archive.json
 Données BRUTES (forecast) sauvegardées dans : weather_stockholm_today+7j.json


Weather_code

- 0 : ciel clair
- 1,2,3 : Partiellement nuageux à couvert
- 51,53,55 : Bruine (légère, modérée dense)
- 61, 63, 65 : pluie (faible, modérée, forte)
- 71,73,75 : chute de neige (legère, modérée, forte)

--> voir pour le ML regroupement par impact ?

| Catégorie | Codes associés | impact théoriques retard |
| --------- | -------------- | ------------------------ |
| Normal | 0,1,2,3 | aucun |
| Précipitations | 51,53,55,61,63,65 | Modéré |
| Risque glace/Neige | 71,73,75 | très fort |

----> Process ETL
